In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists Lakehouse_Demo.Dim_shipmode
(
ShipMode_ID	Long	,
Ship_Mode	string	,
Created_TS	timestamp,
Modified_TS timestamp
)

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date = spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Lakehouse_Demo.Dim_shipmode").first()[0]
print(Max_Date)

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 5, Finished, Available, Finished)

1900-01-01


In [12]:
df_shipmode = spark.sql("select distinct Ship_Mode from Lakehouse_Demo.Sales_Stg where Modified_TS>'{}'".format(Max_Date))
display(df_shipmode.show())

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 14, Finished, Available, Finished)

+--------------+
|     Ship_Mode|
+--------------+
|   First Class|
|      Same Day|
|  Second Class|
|Standard Class|
+--------------+



In [13]:
Max_ID = spark.sql("select coalesce(max(ShipMode_ID),0) from Lakehouse_Demo.Dim_shipmode").first()[0]
print(Max_ID)

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 15, Finished, Available, Finished)

0


In [14]:
df_final = df_shipmode.withColumn("ShipMode_ID",monotonically_increasing_id()+Max_ID+1)

display(df_final.show())

df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 16, Finished, Available, Finished)

+--------------+-----------+
|     Ship_Mode|ShipMode_ID|
+--------------+-----------+
|   First Class|          1|
|      Same Day|          2|
|  Second Class|          3|
|Standard Class|          4|
+--------------+-----------+



In [15]:
%%sql
merge into Lakehouse_Demo.Dim_shipmode as ds 
using ViewShipMode as vs  
on ds.Ship_Mode = vs.Ship_Mode
when MATCHED then 
update set
Modified_TS= current_timestamp()

when not matched then
INSERT
(
ds.ShipMode_ID,
ds.Ship_Mode,
ds.Created_TS,
ds.Modified_TS
)
VALUES
(
vs.ShipMode_ID,
vs.Ship_Mode,
current_timestamp(),
current_timestamp()

)

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 17, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [16]:
%%sql
select * from Lakehouse_Demo.Dim_shipmode

StatementMeta(, 191fa7b1-ce21-45f0-97dd-16c2acd1d899, 18, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>